In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import date
import os

# To avoid unnecessary warnings in the code
import warnings
warnings.filterwarnings('ignore')

In [ ]:
try:
  from google.colab import drive
  IN_COLAB = True
except:
  IN_COLAB = False
  os.chdir("F:\My Drive\Shared\CS 498 Projects\Final Project\Data")
if IN_COLAB:
  drive.mount('/content/drive')
  os.chdir("/content/drive/My Drive/Shared/CS 498 Projects/Final Project/Data")
  os.listdir("/content/drive/My Drive/Shared/CS 498 Projects/Final Project/Data")
df_prep = pd.read_csv('Illinois/Cleaned_Weather/DailyPrep~1950-2019@IL.csv',
                      sep=',', index_col='Date',na_values=-99)
df_tmax = pd.read_csv('Illinois/Cleaned_Weather/DailyTmax~1950-2019@IL.csv',
                      sep=',', index_col='Date',na_values=-99)
df_tmin = pd.read_csv('Illinois/Cleaned_Weather/DailyTmin~1950-2019@IL.csv',
                      sep=',', index_col='Date',na_values=-99)

# Change the index 'Date' format to python datetime.date(yyyy,mm,dd) format
date_index = [date(int(d.split('/')[2]),
                   int(d.split('/')[1]),
                   int(d.split('/')[0])) 
              for d in df_prep.index]
df_prep.set_index(pd.Index(date_index),'Date', inplace=True)
df_tmax.set_index(pd.Index(date_index),'Date', inplace=True)
df_tmin.set_index(pd.Index(date_index),'Date', inplace=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#Creating dataframes with values only between mid-april and mid-october.
APRIL = 4
OCTOBER = 10
req_date_index = []
req_date_index = [i for i,d in enumerate(df_prep.index) if date(d.year,APRIL,15) < d <= date(d.year,OCTOBER,15)]

df_prep_crop = df_prep.iloc[req_date_index,:]
df_tmax_crop = df_tmax.iloc[req_date_index,:]
df_tmin_crop = df_tmin.iloc[req_date_index,:]

In [ ]:
County = [''.join(c.upper().replace('.','').split()) for c in df_prep_crop.columns.values]

for i in range(len(County)):
    if County[i] == 'ROCKISLAND':
        County[i] = 'ROCK ISLAND'
    if County[i] == 'DEWITT':
        County[i] = 'DE WITT'
    if County[i] =='JODAVIESS':
        County[i] = 'JO DAVIESS'
    if County[i] =='STCLAIR':
        County[i] = 'ST. CLAIR'

df_tmax_crop.columns = County
df_tmin_crop.columns = County
df_prep_crop.columns = County

In [ ]:
#Checking the number of missing values for each year of all counties
year = [1950+i for i in range(70)]
df_missing_prep = pd.DataFrame(index=df_prep_crop.columns)
df_missing_tmax = pd.DataFrame(index=df_tmax_crop.columns)
df_missing_tmin = pd.DataFrame(index=df_tmin_crop.columns)
for y in year:
  df_missing_prep[str(y)] = df_prep_crop.loc[date(y,1,1):date(y,12,31),:].isnull().sum().values
  df_missing_tmax[str(y)] = df_tmax_crop.loc[date(y,1,1):date(y,12,31),:].isnull().sum().values
  df_missing_tmin[str(y)] = df_tmin_crop.loc[date(y,1,1):date(y,12,31),:].isnull().sum().values

df_missing_prep.to_csv('Illinois/Cleaned_Weather/missing_prep.csv')
df_missing_tmax.to_csv('Illinois/Cleaned_Weather/missing_tmax.csv')
df_missing_tmin.to_csv('Illinois/Cleaned_Weather/missing_tmin.csv')

In [ ]:
#Defining a function to handle the missing values in precipitation.
def handle_missing_prep(cur_date, all_date, county_name):
  year = list(set([d.year for d in all_date]))
  cur_year = cur_date.year
  num_miss = df_missing_prep.loc[county_name, str(cur_year)]
  year_index = year.index(cur_year)
  if num_miss != 0:
    if year_index >= 5:
      req_dates = []
      for i in range(year_index-5,year_index):
        for day in range(32):
          try:
            req_dates.append(date(year[i],cur_date.month, day))
          except:
            pass
    else:
      req_dates = []
      for i in range(0,year_index+1):
        for day in range(32):
          try:
            req_dates.append(date(year[i],cur_date.month, day))
          except:
            pass
    return req_dates
  else:
    return cur_date

for county in df_prep_crop.columns:
  for d in df_prep_crop.index:
    if np.isnan(df_prep_crop.loc[d, county]):
      df_prep_crop.loc[d, county] = np.mean(df_prep_crop.loc[handle_missing_prep(d, df_prep.index, county),county])


In [ ]:
#Checking the number of missing values for each year of all counties after replacing. 
df_missing_prep_ = pd.DataFrame(index=df_prep_crop.columns)
for y in year:
  df_missing_prep_[str(y)] = df_prep_crop.loc[date(y,1,1):date(y,12,31),:].isnull().sum().values
df_missing_prep_.sum(axis=1).values,df_missing_prep.sum(axis=1).values

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([   4,  436,   62,   33,   16,  570,   15,    1,    0,    5,    0,
          34,   20,    0,    0,    0,    0,    1,   71,  619,   35,    5,
         575, 1176,    0,    0,    7,    3,    3,    0,  205,   14,    0,
        1304,    0,  343,    5,    0,    0,    0,   47,    0,   22,    0,
        1459,    0,    0,    1,    0,    0,    0,    8,    0,    0,    0,
           0,    5,    0,   22,   21,   39,    0,    0,    0,    0,  561,
           0,   11,   20,    1,    0,    0,   16,    0,   73,   23,  219,
         249,    0,    0,  377,    0,    0,  130,  737,    0,    0,  159,
           0,    1,  450,    0,  811,   25,    3,    6,  139

In [ ]:
!pip install geopandas
import geopandas as gpd
gdf = gpd.read_file('Illinois/GIS Files/Illinois_Counties_GCS_WGS84.shp')
adjacent={}
centroid=[]
for i in range(len(gdf.index)):
 centroid.append(gdf.iloc[i].geometry.centroid.y)
df_county_lat = pd.DataFrame({'Latitude': centroid}, index=gdf.iloc[:].COUNTY_NAM)

for County_ in df_county_lat.index: 
  County_lat_diff = pd.DataFrame(index=pd.Index(['Distance']))
  for County in df_county_lat.index:
      County_lat_diff[County] = [abs(df_county_lat.loc[County,'Latitude'] - df_county_lat.loc[County_,'Latitude'])]

  adjacent[County_] = list(County_lat_diff.T.sort_values(by=['Distance']).iloc[1:6].index)


     |████████████████████████████████| 931kB 4.6MB/s 
     |████████████████████████████████| 10.4MB 14.8MB/s 
     |████████████████████████████████| 14.7MB 251kB/s 


In [ ]:
#Handling missing data in minimum temperature and maximum temperature.
for county in df_tmax_crop.columns:
  adjacent_counties = adjacent[county]
  for d in df_tmax_crop.index:
    if np.isnan(df_tmax_crop.loc[d, county]):
      for adjacent_county in adjacent_counties:
        if not np.isnan(df_tmax_crop.loc[d, adjacent_county]):    
          df_tmax_crop.loc[d, county] = df_tmax_crop.loc[d, adjacent_county]
          break
    if np.isnan(df_tmin_crop.loc[d, county]):
      for adjacent_county in adjacent_counties:
        if not np.isnan(df_tmin_crop.loc[d, adjacent_county]):    
          df_tmin_crop.loc[d, county] = df_tmin_crop.loc[d, adjacent_county]
          break
  

In [ ]:
df_missing_tmax_ = pd.DataFrame(index=df_tmax_crop.columns)
df_missing_tmin_ = pd.DataFrame(index=df_tmin_crop.columns)
for y in year:
  df_missing_tmax_[str(y)] = df_tmax_crop.loc[date(y,1,1):date(y,12,31),:].isnull().sum().values
  df_missing_tmin_[str(y)] = df_tmin_crop.loc[date(y,1,1):date(y,12,31),:].isnull().sum().values

In [ ]:
#Checking the number of missing values of minimum temperature for each year of all counties after replacing. 
df_missing_tmin_.sum(axis=1).values,df_missing_tmin.sum(axis=1).values                                                                                                                                                                                                                                                                                                                                                                                              

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([   0,    0,    5,  524,    0,   24,    0,    0,   13,  772,    0,
         727,    0,   36,   70,    0,    0,    0,  727,  309,    0,    0,
         332, 1184,    1,  230,   26,   19,  568,  754, 1414,   24,    0,
          95,    2,   94,   13,    0,   15,    0,    0,   43,   48,    0,
        1236,    0,    0,    0,    0,    0,   77,    7, 1464,    0,    0,
        1014,    0,  524,    0,    0,  398,    5,    0,    0,  772,    0,
           1,    0,   65,  146,   96,    0,    7,    0,   76,    0,  364,
         188,  155,    1, 1414,    0,    0,  550, 1009,    0,    0,    0,
          11,    0, 1221,    0, 1184,    0,   10,    9,   80

In [ ]:
#Checking the number of missing values of maximum temperature for each year of all counties after replacing. 
df_missing_tmax_.sum(axis=1).values,df_missing_tmax.sum(axis=1).values  

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([   0,    0,    5,  430,    1,   24,    0,    4,    6,  684,    0,
         733,    0,   38,   70,    0,    0,    0,  733,  268,    0,    0,
         332, 1185,    5,  225,   22,    1,  566,  621, 1412,   24,    0,
          79,    2,   78,   16,    0,   15,    0,    0,   35,   41,    0,
        1234,    0,    0,    0,    0,    0,   76,    7,    1,    0,    0,
        1014,    0,  430,    0,    0,  398,    6,    1,    0,  684,    0,
           0,    0,   97,  137,   93,    0,    7,    0,   70,    0,  402,
         188,  173,    1, 1412,    0,    0,  551, 1011,    0,    0,    0,
           4,    0, 1219,    0, 1185,    0,    9,   10,   64

In [ ]:
df_prep_crop.to_csv('Illinois/Cleaned_Weather/Daily_prep@IL_1950-2019_Apr-15_Oct-15.csv')
df_tmax_crop.to_csv('Illinois/Cleaned_Weather/Daily_tmax@IL_1950-2019_Apr-15_Oct-15.csv')
df_tmin_crop.to_csv('Illinois/Cleaned_Weather/Daily_tmin@IL_1950-2019_Apr-15_Oct-15.csv')